<a href="https://colab.research.google.com/github/argalusmp/CH2-PS_Recommendation-System/blob/V/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Collab](https://colab.research.google.com/drive/1d9l2-NXW5traKPQ0j-l4eZ2vSI0mEVvV)

# **Build Recommendation System with Content-Based Filtering**

# Packages

Import Packages



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf

# Import Dataset


In [ ]:
user_dataset = pd.read_csv("./users_dataset.csv")
event_dataset= pd.read_csv("./events_dataset.csv")

In [ ]:
df_user = pd.DataFrame(user_dataset)
df_event = pd.DataFrame(event_dataset)

In [ ]:
print(len(df_user))
print(len(df_event))

2786
2786


In [ ]:
skills_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
skills_encoded = skills_encoder.fit_transform(df_user[['Skills']])

In [ ]:
location_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
location_encoded = location_encoder.fit_transform(df_event[['Location']])


In [ ]:
# One-hot encoding untuk kategori acara
category_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
category_encoded = category_encoder.fit_transform(df_event[['Category']])

In [ ]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(df_volunteer[['Skills', 'Location', 'Age']], df_volunteer['Target_Label'], test_size=0.2, random_state=42)

# Preprocessing Data


In [ ]:
all_data = pd.merge(user_dataset, event_dataset, how='cross')

In [ ]:
# Pisahkan data menjadi train dan test
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)

In [ ]:
# Preprocessing user data
user_dataset['Skills'] = user_dataset['Skills'].str.lower()
user_dataset['Availability'] = user_dataset['Availability'].str.lower()
user_dataset['Location'] = user_dataset['Location'].str.lower()
user_dataset['Type of Organization'] = user_dataset['Type of Organization'].str.lower()

## Preprocessing event data
event_dataset['Kualifikasi'] = event_dataset['Kualifikasi'].str.lower()
event_dataset['Domisili'] = event_dataset['Domisili'].str.lower()
event_dataset['Kategori'] = event_dataset['Kategori'].str.lower()
event_dataset['Age'] = event_dataset['Age'].str.lower()


## Memisahkan user skill menjadi beberapa kolom terpisah untuk one hot
user_skills_split = user_dataset['Skills'].str.split(', ', expand=True)

## Create one-hot encoding for user skills
user_skills_one_hot = pd.get_dummies(user_skills_split, prefix='Skill')

In [ ]:
## Menggabungkan dataset
merged_data = pd.merge(user_dataset, event_dataset, how='cross')
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

## Create one-hot encoding for user and event data
user_one_hot = pd.get_dummies(merged_data[['Age_x', 'Availability', 'Location', 'Type of Organization']], prefix='User')
event_one_hot = pd.get_dummies(merged_data[[ 'Kategori', 'Age_y','Domisili']], prefix='Event')

In [ ]:
## Check Display
pd.set_option('display.max_columns', None)

event_one_hot
#user_one_hot

In [ ]:
## Merge Onehot encoding with data user
user_data_encode = pd.concat([user_one_hot, user_skills_one_hot], axis=1)
user_data_encode

In [ ]:
## Memisahkan event kualifikasi menjadi beberapa kolom terpisah untuk one hot
event_kualifikasi_split = event_dataset['Kualifikasi'].str.split(', ', expand=True)

## Create one-hot encoding for kualifikasi
event_kualifikasi_one_hot = pd.get_dummies(event_kualifikasi_split, prefix='Kualifikasi')

## Merge Kualifikasi with dataset event after one-hot kualifikasi
event_data_encode = pd.concat([event_one_hot, event_kualifikasi_one_hot], axis=1)


In [ ]:
## Check event encode display
event_data_encode

In [ ]:
print(user_data_encode.isnull().sum())
print(event_data_encode.isnull().sum())


In [ ]:
## For set Y to target
#target_columns = ['Kualifikasi_kualifikasi1', 'Kualifikasi_kualifikasi2', ...]
#X_train, X_test, y_train, y_test = train_test_split(user_data_encode, event_data_encode[target_columns], test_size=0.2, random_state=42)



## Train-test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Nyoba proses nilai umur



```
###Pemrosesan Data Umur
def process_age(value):
    if '-' in str(value):  # Jika nilai adalah rentang umur
        age_range = value.split('-')
        return (int(age_range[0]) + int(age_range[1])) / 2
    elif isinstance(value, int):  # Jika nilai adalah umur tunggal dan sudah integer
        return value
    else:
        # Penanganan lainnya
        return None


event_dataset['Age'] = event_dataset['Age'].apply(process_age)

```



# Try and Try and Try

# Pusing NaN

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

# Baca dataset
user_dataset = pd.read_csv("./user_data.csv")
event_dataset = pd.read_csv("./event_data.csv")

# Preprocessing user data
user_dataset['Skills'] = user_dataset['Skills'].str.lower()
user_dataset['Availability'] = user_dataset['Availability'].str.lower()
user_dataset['Location'] = user_dataset['Location'].str.lower()
user_dataset['Type of Organization'] = user_dataset['Type of Organization'].str.lower()

# Preprocessing event data
event_dataset['Kualifikasi'] = event_dataset['Kualifikasi'].str.lower()
event_dataset['Domisili'] = event_dataset['Domisili'].str.lower()
event_dataset['Kategori'] = event_dataset['Kategori'].str.lower()
event_dataset['Age'] = event_dataset['Age'].astype(str).str.lower()

# Gabungkan data
full_data = pd.merge(user_dataset, event_dataset, how='cross')

# Pisahkan data menjadi train dan test
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)

# One-hot encoding dan penggabungan data
user_skills_split = user_dataset['Skills'].str.split(', ', expand=True)
user_skills_one_hot = pd.get_dummies(user_skills_split, prefix='Skill')

user_one_hot_train = pd.get_dummies(train_data[['Age_x', 'Availability', 'Location', 'Type of Organization']], prefix='User')
event_one_hot_train = pd.get_dummies(train_data[['Kategori', 'Age_y', 'Domisili']], prefix='Event')
event_kualifikasi_one_hot_train = pd.get_dummies(train_data['Kualifikasi'].str.split(', ', expand=True), prefix='Kualifikasi')

train_data_encode = pd.concat([user_one_hot_train, user_skills_one_hot, event_one_hot_train, event_kualifikasi_one_hot_train], axis=1)

# Normalisasi data
scaler = StandardScaler()
train_data_normalize = scaler.fit_transform(train_data_encode)

In [ ]:
train_data_normalize = np.nan_to_num(train_data_normalize, nan=np.nanmean(train_data_normalize, axis=0))


In [ ]:
print(np.isnan(train_data_normalize).any())


False


In [ ]:
# Hitung similarity matrix menggunakan cosine similarity
similarity_matrix = cosine_similarity(train_data_normalize, train_data_normalize)

In [ ]:
print(f"Cosine Similarity: {similarity_matrix[0][0]}")

# This one using vectorize


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


# Load the datasets
user_df = pd.read_csv('users_dataset.csv')
event_df = pd.read_csv('events_dataset.csv')

# Preprocessing
user_df['Skills'] = user_df['Skills'].apply(lambda x: ' '.join(x.lower().split(', ')) if pd.notnull(x) else '')
event_df['Qualifications'] = event_df['Qualifications'].apply(lambda x: ' '.join(x.lower().split(', ')) if pd.notnull(x) else '')

# Vectorize the skills and qualifications
vectorizer = TfidfVectorizer(stop_words='english')
user_matrix = vectorizer.fit_transform(user_df['Skills'])
event_matrix = vectorizer.transform(event_df['Qualifications'])

# Compute the cosine similarity
cosine_sim = linear_kernel(user_matrix, event_matrix)

# Function to get recommendations
def get_recommendations(user_index, cosine_sim=cosine_sim):
    # Get the pairwsie similarity scores of all events for that user
    sim_scores = list(enumerate(cosine_sim[user_index]))

    # Sort the events based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar events
    sim_scores = sim_scores[0:10]

    # Get the event indices
    event_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar events
    return event_df['Event_id'].iloc[event_indices]

# Test the system relation user 1 (index 0) to event
print(get_recommendations(0))


2592    E_2593
2683    E_2684
2723    E_2724
2735    E_2736
2744    E_2745
2531    E_2532
2659    E_2660
2668    E_2669
2776    E_2777
2399    E_2400
Name: Event_id, dtype: object
